Übung: Verteilung 
-----------------

In dieser Übung erstellen wir mehrere Pods (Instanzen) ab dem gleichen Image (Applikation) mit jeweils einem ReplicaSet, Deployment und Service.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [ ]:
! kubectl create namespace rs

Erzeugen eines Deployments, hier der das Beispiel von Docker mit einem Web Server welche die aktuelle IP-Adresse ausgibt.

In [ ]:
! # kubectl run apache --image=registry.gitlab.com/mc-b/misegr/hello-world --namespace rs",
! kubectl create deployment apache --image=registry.gitlab.com/mc-b/misegr/hello-world --namespace rs

Ausgabe der Erzeugten Ergebnisse und die YAML Datei welche den Erzeugten Ressourcen beschreibt.

Ab `spec.containers` kommt erst der Pod.

In [ ]:
! kubectl get pod,deployment,replicaset,service --namespace rs

In [ ]:
! kubectl get deployment apache -o yaml --namespace rs | kubectl-neat

Um die ReplicaSet Funktionalität zu Demonstrieren, setzen wir die Anzahl der laufenden Pods auf 3 und schauen uns das Ergebnis an

In [ ]:
! kubectl --namespace rs scale --replicas=3 deployment/apache  
! kubectl get pod,deployment,replicaset,service --namespace rs

Zu dem **Deployment** `apache` Erzeugen wir einen Service. Dadurch wird der Web Server von aussen sichtbar.

Der Port 80 wird von Kubernetes automatisch auf den nächsten freien Port gemappt.

Das Ergebnis schauen wir uns wieder an.

In [ ]:
! kubectl expose deployment/apache --type="LoadBalancer" --port 80 --namespace rs
! kubectl get pod,deployment,replicaset,service,endpoints -o wide --namespace rs

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port als URL aufbereiten.

In [ ]:
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service --namespace rs apache -o=jsonpath='{ .spec.ports[0].nodePort }')/"


Wird der obige Link in mehreren Browsern geöffnet sollte jeweils ein anderer Hostname erscheinen. 

**Erklärung**: Kubernetes verteilt die Anfragen auf die laufenden Pods.

Zum Aufräumen genügt es den Namespace zu löschen

In [ ]:
! kubectl delete namespace rs